In [12]:
# https://rosettacode.org/wiki/Sorting_algorithms/Heapsort#Python

import numpy as np
from line_profiler import LineProfiler
from pysr import PySRRegressor
import random
import os
import sys
import math
import pickle

## Heapsort Algorithm

In [13]:
def heapsort(lst): # N log N 
  ''' Heapsort. Note: this function sorts in-place (it mutates the list). '''

  # in pseudo-code, heapify only called once, so inline it here
  for start in range(int((len(lst)-2)/2), -1, -1):
    siftdown(lst, start, len(lst)-1)

  for end in range(int(len(lst)-1), 0, -1):
    lst[end], lst[0] = lst[0], lst[end]
    siftdown(lst, 0, end - 1)

def siftdown(lst, start, end):  
  root = start
  while True:
    child = root * 2 + 1
    if child > end: break
    if child + 1 <= end and lst[child] < lst[child + 1]:
      child += 1
    if lst[root] < lst[child]:
      lst[root], lst[child] = lst[child], lst[root]
      root = child
    else:
      break

## Frequency Count Method

In [14]:
X_y = []
i=50
for n in range(2,i+1): # started with 2 itens until 50 itens in the list
  lprofiler = LineProfiler()
  lp_wrapper = lprofiler(heapsort)

  # create a list with random number between 1 to 10000
  # with n number in the list
  input = random.choices(range(10000), k=n)
  lp_wrapper(input)

  stats = lprofiler.get_stats()
  line_numbers = []
  hits = []

  for line in stats.timings.values():
    for i in line:
      line_numbers.append(i[0])
      hits.append(i[1])

  X_y.append([n, sum(hits)])

dados = np.array(X_y)

X = dados[:, 0]
y = dados[:, 1]
X_reshaped = X.reshape(-1, 1)

In [15]:
resultados_com_menor_loss = []
repeat = 5
registros = []

original_stdout = sys.stdout

with open(os.devnull, 'w') as devnull:
  sys.stdout = devnull

  for i in range(repeat):

    # first combination
    reg1 = PySRRegressor(
      binary_operators=["*", "+"],
      unary_operators=["log", "square", "cube"],
    )

    fit1 = reg1.fit(X_reshaped, y)
    best_program1 = fit1.get_best()

    registro1 = []
    for index, value in enumerate(best_program1):
      registro1.append(value)
      
    registros.append(registro1)

    # second combination
    reg2 = PySRRegressor(
      binary_operators=["*"],
      unary_operators=["log", "square", "cube"],
    )

    fit2 = reg2.fit(X_reshaped, y)
    best_program2 = fit2.get_best()

    registro2 = []
    for index, value in enumerate(best_program2):
      registro2.append(value)
    registros.append(registro2)

    # third combinarion
    reg3 = PySRRegressor(
      binary_operators=["+"],
      unary_operators=["log", "square", "cube"],
    )

    fit3 = reg3.fit(X_reshaped, y)
    best_program3 = fit3.get_best()

    registro3 = []
    for index, value in enumerate(best_program3):
      registro3.append(value)
    registros.append(registro3)
    
sys.stdout = original_stdout

[ Info: Started!
0.0%┣                                              ┫ 0/600 [00:00<00:-7, -0s/it]Expressions evaluated per second: [.....]. Head worker occupation: 0.0%         Press 'q' and then <enter> to stop execution early.                             Hall of Fame:                                                                   ---------------------------------------------------------------------------------------------------                                                             Complexity  Loss       Score     Equation                                       1           1.332e+04  1.594e+01  y = 1.9303                                    3           3.866e+03  6.183e-01  y = (1.8551 * x₀)                             5           7.566e-01  4.269e+00  y = ((2.0336 * x₀) * 1.9452)                  ---------------------------------------------------------------------------------------------------
0.3%┣▏                                              ┫ 2/600 [00:00<01:19, 8it/s]Exp

In [16]:
registros_ = registros

In [17]:
for i in registros_:
  loss = i[1]
  score = i[2]
  complexity = i[0]
  w = (loss * score)/complexity
  if math.isnan(w):
    i.append(0)
  else:
    i.append(w)

lista_melhor_valor = max(registros_, key=lambda x: x[6])

## Save result

In [18]:
def salvar_dados(dados, arquivo):
  if os.path.exists(arquivo):
    with open(arquivo, 'rb') as f:
      dados_exist = pickle.load(f)

    dados_exist.update(dados)
  else:
    dados_exist = dados
  
  with open(arquivo, 'wb') as f:
    pickle.dump(dados_exist, f)
  
novos_dados = {'heapsort_NlogN': lista_melhor_valor[4]}
caminho_arquivo = 'dados.pickle'

salvar_dados(novos_dados, caminho_arquivo)

In [19]:
def carregar_dados(arquivo):
    # Carrega os dados do arquivo pickle
    with open(arquivo, 'rb') as f:
        dados = pickle.load(f)
    return dados

caminho_arquivo = 'dados.pickle'
dados_carregados = carregar_dados(caminho_arquivo)

print("Conteúdo do arquivo pickle:")
for k, v in dados_carregados.items(): print(k, ' = ',v)

Conteúdo do arquivo pickle:
shellsort_NlogN  =  4.862569*x0*log(x0)
mergesort_NlogN  =  5.5040717*x0 + 8.03698770358715
bubblesort_x^2  =  1.517833856016*x0**2
levenshtein_distance_MN  =  3.52552934090449*x1**2
selectsort_x^2  =  1.13491224207076*x0**2
insertionsort_x^2  =  0.84217347354001*x0**2
quicksort_x^2  =  7.605706*x0*log(x0)
linear_search_x  =  2.0593596*x0
binary_search_logx  =  log(x0)**2 + 13.966311
heaosort_NlogN  =  4*x0 - 1.4897914
